# 12 March, 2024
# CSC354 – Assignment1 – ML – Concept Learning
# Muhammad Umar Asif
# FA21-BSE-075
# It is the implementation of candidate elimination algorithm to find the set of all hypothesis consistent with the training examples.

In [35]:
import random
import csv

In [36]:
data = [('big', 'red', 'circle', 'N'),
 ('small', 'red', 'triangle', 'N'),
 ('small', 'red', 'circle', 'Y'),
 ('big', 'blue', 'circle', 'N'),
 ('small', 'blue', 'circle', 'Y')]

In [38]:
def attribute_values(exp):
    d = [set() for i in exp[0]]
    for x in exp:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]

attribute_values(data)

[['big', 'small'], ['blue', 'red'], ['circle', 'triangle'], ['N', 'Y']]

In [39]:
def general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)

def fulfills(exp, hypothesis):
    return general(hypothesis, exp)

def minimal_g(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not fulfills(x[i:i+1], h[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]

def minimal_s(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

In [41]:
def g_null(n):
    return ("?",)*n

def s_null(n):
    return ('0',)*n

def candidate_elimination(examples):
    values = attribute_values(examples)[:-1]

    G = set([g_null(len(values))])
    S = set([s_null(len(values))])
    i=0
    print("\n G[{0}]:".format(i),G)
    print("\n S[{0}]:".format(i),S)
    for xcx in examples:
        i=i+1
        x, cx = xcx[:-1], xcx[-1]
        if cx=='Y':
            G = {g for g in G if fulfills(x, g)}
            S = generalize_S(x, G, S)
        else: # x is negative example
            S = {s for s in S if not fulfills(x, s)}
            G = specialize_G(x, values, G, S)
        print("\n G[{0}]:".format(i),G)
        print("\n S[{0}]:".format(i),S)
    return

def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not fulfills(x, s):
            S.remove(s)
            Splus = minimal_g(s, x)
            S.update([h for h in Splus if any([general(g,h)
                                               for g in G])])
            S.difference_update([h for h in S if
                                 any([general(h, h1)
                                      for h1 in S if h != h1])])
    return S

def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if fulfills(x, g):
            G.remove(g)
            Gminus = minimal_s(g, domains, x)
            G.update([h for h in Gminus if any([general(h, s)
                                                for s in S])])
            G.difference_update([h for h in G if
                                 any([general(g1, h)
                                      for g1 in G if h != g1])])
    return G

candidate_elimination(data)


 G[0]: {('?', '?', '?')}

 S[0]: {('0', '0', '0')}

 G[1]: {('?', '?', 'triangle'), ('?', 'blue', '?'), ('small', '?', '?')}

 S[1]: {('0', '0', '0')}

 G[2]: {('?', 'blue', '?'), ('small', '?', 'circle'), ('big', '?', 'triangle')}

 S[2]: {('0', '0', '0')}

 G[3]: {('small', '?', 'circle')}

 S[3]: {('small', 'red', 'circle')}

 G[4]: {('small', '?', 'circle')}

 S[4]: {('small', 'red', 'circle')}

 G[5]: {('small', '?', 'circle')}

 S[5]: {('small', '?', 'circle')}
